In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import pmdarima as pm

ModuleNotFoundError: No module named 'pmdarima'

In [4]:
import warnings
# 한글설치
plt.rc('font', family = 'Malgun Gothic')

# 음수표시
plt.rc('axes', unicode_minus = False)

# 실행결과 경고메세지 출력제외
warnings.filterwarnings('ignore')

In [ ]:
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

In [ ]:
df['주문일시'] = pd.to_datetime(df['주문일시'])

In [ ]:
ar = pd.pivot_table(data=df_smp, index='주문일시', columns='물품중분류', values='구매수량', aggfunc='size')

In [ ]:
ar = ar.fillna(0)
ar

In [ ]:
df_2weeks = ar.resample('2W').sum()

In [ ]:
df_2weeks

In [ ]:
s = pd.DataFrame(df_2weeks.sum()).transpose()

In [ ]:
for i in s.columns:
    if i == 0:
        df_2weeks.drop()

df_2weeks

In [ ]:
import statsmodels.tsa.api as tsa
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
from pylab import rcParams

## 이건 예시(arima)

forcast = pd.DataFrame()
model = ARIMA(df_2weeks['가공소금'], order=(0,1,1))
model_fit = model.fit()
forecast = model_fit.forecast(steps=4)
forecast.columns = ['date', 'forecast_quantity']
forecast['product'] = '가공소금'
pd.DataFrame(forecast)

In [ ]:
df_or = df[df['유기신선'].str.contains('유기농')]

In [ ]:
df_pm = df_or.groupby('물품중분류')['구매수량'].sum()
df_pm

In [ ]:
df_pm = df_pm.sort_values(by = '구매수량', ascending=False).head(30)

In [ ]:
df_pm = df_pm['구매수량'].reset_index()
df_pm

### 애호박 예시

In [2]:
a = df_2weeks['애호박']

NameError: name 'df_2weeks' is not defined

In [ ]:
from statsmodels.tsa.stattools import adfuller

adfuller(a, autolag='AIC')

#### ADF 검정

In [ ]:
def ADF_test(data):
    results = adfuller(data, autolag='AIC')
    s = results[0]
    p = results[1]
    cv = results[4]

    print('-' * 30)
    print('Augemented Dicket-Fuller Test')
    print('H0 : 단위근이 존재한다(비정상 시계열)')
    print('H1 : 단위근이 없다(정상 시계열)')
    print('Critical Values : {}'.format(cv))
    print('-'* 30)
    print('Test Statistic : {:.4f}'.format(s))
    print('p-value : {:.4f}'.format(p))
    print('-'*30)

In [ ]:
ADF_test(a)

In [ ]:
df_diff = a - a.shift()
df_diff

In [ ]:
def plot_df(data, color, alpha, label):
    plt.figure(figsize=(11,5))
    plt.plot(data, color=color, alpha=alpha, label=label)
    plt.title('Mean Temperature of Bindukuri')
    plt.ylabel('Mean temperature')
    plt.legend()
    plt.show()

In [ ]:
plot_df(df_diff, color='blue', alpha=0.25, label='differencing')

In [ ]:
def plot_rolling(data, roll_size):
    roll_mean = data.rolling(window=roll_size).mean()
    roll_std = data.rolling(window=roll_size).std()

    plt.figure(figsize=(11, 5))
    plt.plot(data, color='blue', alpha=0.25, label='Original')
    plt.plot(roll_mean, color='black', label='Rolling mean')
    plt.plot(roll_std, color='red', label='Rolling std')
    plt.title('Rolling Mean & Standard Deviation')
    plt.ylabel('Mean Temperature')
    plt.legend()
    plt.show()

plot_rolling(df_diff, 2)

#### 비정상 시계열일 경우

In [ ]:
df_diff2 = a.diff().dropna()

In [ ]:
ADF_test(df_diff2)

In [ ]:
train = a.loc['2022-01-23':'2022-09-04']
test = a.loc['2022-09-18':'2022-11-13']

print('train data 개수 : ', len(train))
print('test data 개수 : ', len(test))

In [ ]:
import pmdarima as pm

kpss_diffs = pm.arima.ndiffs(train, alpha=0.05, test='kpss', max_d=5)
adf_diffs = pm.arima.ndiffs(train, alpha=0.05, test='adf', max_d=5)
n_diffs = max(kpss_diffs, adf_diffs)

print(f"Optimized 'd' = {n_diffs}")

In [ ]:
auto_arima_model = pm.auto_arima(y=train,
                                d=n_diffs,
                                start_p=0,
                                max_p=5,
                                start_q=0,
                                max_q=5,
                                m=1,
                                seasonal=False,
                                stepwise=True,
                                trace=True)

In [ ]:
model2 = pm.auto_arima(train, d=2, seasonal=False, trace=True)

In [ ]:
model2.summary()

In [ ]:
def one_step():
    fc, conf = model2.predict(n_periods=1, return_conf_int=True)
    return fc.tolist()[0], np.asarray(conf).tolist()[0]

In [ ]:
y_pred = []

for new_ob in test:
    fc, conf = one_step()
    y_pred.append(fc)
    model2.update(new_ob)

### 학습 확인 지표

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
print(f'MAPE: {mean_absolute_percentage_error(test, y_pred):.3f}')

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(12, 8))
# for i, j in enumerate(df_pm['물품중분류']):
print(sns.lineplot(data=ar, x='주문일시', y='스낵', ax=axes[0,0]))

print(sns.lineplot(data=ar, x='주문일시', y='콩나물', ax=axes[1,0]))

print(sns.lineplot(data=ar, x='주문일시', y='김가공', ax=axes[2,0]))

print(sns.lineplot(data=ar, x='주문일시', y='과즙', ax=axes[0,1]))

print(sns.lineplot(data=ar, x='주문일시', y='양파', ax=axes[1,1]))
print(sns.lineplot(data=ar, x='주문일시', y='오이', ax=axes[2,1]))
print(sns.lineplot(data=ar, x='주문일시', y='식빵, 식사대용', ax=axes[0,2]))
print(sns.lineplot(data=ar, x='주문일시', y='당근', ax=axes[1,2]))
print(sns.lineplot(data=ar, x='주문일시', y='가래떡', ax=axes[2,2]))